# Assignment 1: Fine-tuning for text classification of Japanese review

- Choose the appropriate pre-trained model for fine-tuning.
- Examine the performance of the fine-tuned model on the test dataset.
- Use the fine-tuned model to infer the sentiment of raw text.

```
text1="使い物にならない"
text2="とても気に入った"
```

In [1]:
!nvidia-smi

Fri Jun  6 14:25:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:01:00.0  On |                  Off |
| 30%   39C    P8             20W /  300W |    2504MiB /  49140MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [9]:
from datasets import load_dataset
from datasets import load_dataset, DatasetDict

dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
})

In [8]:
# Shuffle the dataset
dataset = dataset.shuffle(seed=42)

# Subset the dataset
train_dataset = dataset['train'].select(range(3000))
valid_dataset = dataset['train'].select(range(3000, 3800))
test_dataset = dataset['train'].select(range(3800, 4600))

# Compile them into a DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'validation': valid_dataset,
    'test': test_dataset
})

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 800
    })
})

In [5]:
dataset.set_format(type="pandas")
train_df = dataset["train"][:]
train_df.head(5)

,text,source,label
0,4つ購入したが、2つは使用できず。接触が悪いのかわからないが、2度と購入しません,amazon_reviews_multi,2
1,キャンプでカセットコンロを使用しています。 キャンプでちょっと変わった事がしたくて購入！笑 ...,amazon_reviews_multi,0
2,こちらのiTunesカード200＄分を購入しました。 50＄４枚購入したはずが、届いたのは5...,amazon_reviews_multi,2
3,macbook pro 13インチと書いてあったので書いてあったので購入しましたが2016l...,amazon_reviews_multi,1
4,安かろう悪かろうです。 TVはが悪く暗く自動サーチなしでいちいちサーチしなおさなければダメ。...,amazon_reviews_multi,2


In [6]:
dataset.reset_format()